In [1]:
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import KNeighborsClassifier
import sklearn
from __future__ import print_function
import librosa
import numpy as np
import matplotlib.pyplot as plt
from librosa import display,core
from dtw import dtw
from numpy import linalg as LA
from math import sqrt
from scipy import stats
from sklearn import svm
from sklearn.svm import SVR
from mlxtend.plotting import plot_decision_regions
import matplotlib.patches as mpatches
from sklearn.model_selection import cross_val_predict, cross_val_score,cross_validate
from sklearn import metrics
from mlxtend.plotting import plot_confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

from collections import Counter

from imblearn.over_sampling import RandomOverSampler
#%matplotlib inline

In [2]:
import librosa
import numpy as np
from sklearn.model_selection import train_test_split
import os

def createTrainingData(start, end):
    samples = []
    telangana_dir = 'audio/Telangana'
    coastal_dir = 'audio/Coastal'
    rayalseema_dir = 'audio/Rayalseema'
    
    regions = {
        'Telangana': telangana_dir,
        'Coastal': coastal_dir,
        'Rayalseema':rayalseema_dir
    }

    for region, region_dir in regions.items():
        if region == 'Telangana':
            file_range = range(239, 357)
        elif region == 'Rayalseema':
            file_range = range(116, 215)
        elif region == 'Coastal':
            file_range = range(1, 102)  # Assuming audio files in Coastal are named from '1.wav' to '101.wav'

        for i in file_range:
            ind = 0
            try:
                file_path = os.path.join(region_dir, f"{i}.wav")
                a, s = librosa.load(file_path)
            except:
                ind = 1

            if ind == 0:
                mfcc = librosa.feature.mfcc(y=a, sr=s)
                temp = mfcc.T[1][start:end]
                for frame in range(10, 50):
                    temp = np.concatenate((temp, mfcc.T[frame][start:end]))
                samples.append([temp, region])

    print(len(samples))
    X = [i[0] for i in samples]
    Y = [j[1] for j in samples]
    return X, Y



In [3]:
clf4 = svm.SVC()
data,label = createTrainingData(1,19)
X_train, X_test, y_train, y_test = train_test_split(data, label, test_size = 0.2)

C:\Users\Lavanya Tetakali\AppData\Local\Temp\ipykernel_5836\230047187.py:30: UserWarning: PySoundFile failed. Trying audioread instead.
  a, s = librosa.load(file_path)
C:\Users\Lavanya Tetakali\anaconda3\lib\site-packages\librosa\core\audio.py:183: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


317


In [4]:
Counter(label)

Counter({'Telangana': 118, 'Coastal': 100, 'Rayalseema': 99})

In [5]:
X_train, X_test, y_train, y_test  = train_test_split(data,label)
ros = RandomOverSampler(random_state=0)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)
print(sorted(Counter(y_resampled).items()))

[('Coastal', 81), ('Rayalseema', 81), ('Telangana', 81)]


In [6]:
from sklearn.svm import SVC

def predict_svm(X, Y, Z):
    svm = SVC()
    svm.fit(X, Y)
    prediction = svm.predict(Z)
    scores = cross_val_score(svm, data, label)  # You should have 'data' and 'label' defined somewhere in your code.
    print(scores.mean())
    return prediction

def printPrediction(prediction,actual_label):
    print("1 -> Telangana, 0-> Coastal, 2-> Rayalseema\n")
    for i in range(len(prediction)):
        print("Actual ->",actual_label[i]," Prediction ->",prediction[i])

def accuracy(prediction,actual):
    count=0
    l=len(prediction)
    for i in range(l):
        if(prediction[i]==actual[i]):
            count+=1
    return count/l*100

In [7]:
clf = svm.SVC()
clf.fit(X_resampled,y_resampled)
y_pred=clf.predict(X_test)
print("Accuracy:",accuracy(y_pred,y_test))

Accuracy: 95.0


In [8]:
conf_matrix = confusion_matrix(y_test, y_pred)
conf_matrix

array([[18,  0,  1],
       [ 0, 20,  3],
       [ 0,  0, 38]], dtype=int64)

In [9]:
# Access the elements of the confusion matrix
tn = conf_matrix[0, 0]  # True negatives
fp = conf_matrix[0, 1]  # False positives
fn = conf_matrix[1, 0]  # False negatives
tp = conf_matrix[1, 1]  # True positives

In [10]:
precision = round(tp/(tp+fp),4)
recall = round(tp/(tp+fn),4)
fmeasure = round(2*((precision*recall)/(precision+recall)),4)
rej = round(tn/(tn+fp),4)
acc = round((tn+tp)/(tn+tp+fp+fn),4)

In [11]:
print(precision,recall,fmeasure,rej,acc)

1.0 1.0 1.0 1.0 1.0
